<a href="https://colab.research.google.com/github/James-R-Chapman/Kia-Invoices/blob/main/dealertrack_variance_to_xlsx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.1 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140080 sha256=2ea58b17653ea08da35a3ca1945f28a512c69a3a075c6b8d861e6a7708750d01
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.8 MB/s eta 0:00:00


In [ ]:
import io
import csv
import re
import pandas as pd
import pdfminer
import pdfminer.high_level
import pdfminer.layout
from google.colab import files
import os

# Upload the PDF file
uploaded = files.upload()

# Get the uploaded PDF file name
pdf_file_name = next(iter(uploaded))

# Convert PDF to text
txt_file_path = '/content/Report.txt'
with open(txt_file_path, 'w') as output_file:
    output_file.write(pdfminer.high_level.extract_text(io.BytesIO(uploaded[pdf_file_name])))

# Read the text from the file
with open(txt_file_path, 'r') as f:
    text = f.read()

# Find the index of the first line that contains "Part Number/Description"
start_index = 0
for i, line in enumerate(text.split('\n')):
    if "Part Number/Description" in line:
        start_index = i + 2  # Start 2 lines below the line containing "Part Number/Description"
        break

# Split the text into lines
lines = text.split('\n')

# Open a new CSV file for writing
csv_file_path = 'output.csv'
with open(csv_file_path, 'w', newline='') as f:
    writer = csv.writer(f)

    # Write the column headers
    writer.writerow(
        ['Page', 'Part Number', 'Description', 'Sheet', 'Y/P Bin', 'Sys Bin', 'Phy Count', 'Phy Y/P', 'Phy Tot', 'Comp',
         'Var', 'Unit Cost', 'Dollar Variance', 'Open R/O', 'Total Value'])

    # Loop over the lines in the text
    current_page = 1  # Initialize current_page to 1
    for line in lines[start_index:]:
        # Extract the page number from the line using a regular expression
        match = re.search(r'Page\s+(\d+)', line)
        if match:
            current_page = match.group(1)
            continue

        # Extract the fields from the line based on their character count
        if ':' in line:
            parts = line.split(':', 1)
            part_num = parts[0].strip()
            if len(parts) > 1:
                description = parts[1].strip()[:25]  # Use the first 40 characters of the description
            else:
                description = ''
        else:
            part_num = ''
            description = ''

        fields = [
            current_page,
            part_num,
            description,
            line[43:49].strip(),  # Sheet
            line[49:57].strip(),  # Y/P Bin
            line[57:70].strip(),  # Sys Bin
            line[70:80].strip(),  # Phy Count
            line[80:86].strip(),  # Phy Y/P
            line[86:93].strip(),  # Phy Tot
            line[93:100].strip(),  # Comp
            line[100:107].strip(),  # Var
            line[107:118].strip(),  # Unit Cost
            line[118:130].strip(),  # Dollar Variance
            line[130:138].strip(),  # Open R/O
            line[138:].strip()  # Total Value
        ]

        # Write the fields to the CSV file
        writer.writerow(fields)

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Define the regex pattern
pattern = r'PD3350R'

# Filter rows based on conditions
df = df[~df['Part Number'].str.contains(pattern, na=False, regex=True)]
df = df[~df['Sheet'].str.startswith(('='), na=False)]
df = df[~df['Phy Count'].str.startswith('=', na=False)]
df = df[~df['Total Value'].str.startswith('="', na=False)]

# Additional conditions
df = df[~((df['Sheet'].str.contains('Sheet', na=False)) &
          (df['Y/P Bin'].str.contains('Y/P Bi', na=False)) &
          (df['Sys Bin'].str.contains('n    Sys Bin', na=False)) &
          (df['Phy Count'].str.contains('Count', na=False)) &
          (df['Phy Y/P'].str.contains('Y/P', na=False)) &
          (df['Phy Tot'].str.contains('Tot', na=False)) &
          (df['Comp'].str.contains('Comp', na=False)) &
          (df['Var'].str.contains('Var', na=False)) &
          (df['Unit Cost'].str.contains('Cost', na=False)) &
          (df['Dollar Variance'].str.contains('Variance', na=False)) &
          (df['Open R/O'].str.contains('R/O', na=False)) &
          (df['Total Value'].str.contains('Value', na=False)))]

# Add an additional condition
df = df[~((df['Part Number'].isna()) &
          (df['Description'].isna()) &
          (df['Sheet'].isna()) &
          (df['Y/P Bin'].isna()) &
          (df['Sys Bin'].isna()) &
          (df['Phy Count'].isna()) &
          (df['Phy Y/P'].isna()) &
          (df['Phy Tot'].isna()) &
          (df['Comp'].isna()) &
          (df['Var'].isna()) &
          (df['Unit Cost'].isna()) &
          (df['Dollar Variance'].isna()) &
          (df['Open R/O'].isna()) &
          (df['Total Value'].isna()))]

df = df[~((df['Part Number'].isna()) &
          (df['Description'].isna()) &
          (df['Sheet'].isna()) &
          (df['Y/P Bin'].isna()) &
          (df['Sys Bin'].isna()) &
          (df['Comp'].isna()) &
          (df['Var'].isna()) &
          (df['Unit Cost'].str.contains('Unit', na=False)) &
          (df['Dollar Variance'].str.contains('Dollar', na=False)) &
          (df['Open R/O'].str.contains('Open', na=False)) &
          (df['Total Value'].str.contains('Total', na=False)))]

# Convert to XLSX and save the file with the same name as the uploaded file
xlsx_file_name = os.path.splitext(pdf_file_name)[0] + '.xlsx'
xlsx_file_path = os.path.join('/content/', xlsx_file_name)
df.to_excel(xlsx_file_path, index=False)

print(f"CSV file saved: {csv_file_path}")
print(f"XLSX file saved: {xlsx_file_path}")


Saving 2ND UNIT VARIANCE DETAIL.PDF to 2ND UNIT VARIANCE DETAIL.PDF
CSV file saved: output.csv
XLSX file saved: /content/2ND UNIT VARIANCE DETAIL.xlsx


WORKING ABOVE


In [ ]:
import io
import csv
import re
import pandas as pd
import pdfminer
import pdfminer.high_level
import pdfminer.layout
from google.colab import files
import os

# Upload the PDF file
uploaded = files.upload()

# Get the uploaded PDF file name
pdf_file_name = next(iter(uploaded))

# Convert PDF to text
txt_file_path = '/content/Report.txt'
with open(txt_file_path, 'w') as output_file:
    output_file.write(pdfminer.high_level.extract_text(io.BytesIO(uploaded[pdf_file_name])))

# Read the text from the file
with open(txt_file_path, 'r') as f:
    text = f.read()

# Find the index of the first line that contains "Part Number/Description"
start_index = 0
for i, line in enumerate(text.split('\n')):
    if "Part Number/Description" in line:
        start_index = i + 2  # Start 2 lines below the line containing "Part Number/Description"
        break

# Split the text into lines
lines = text.split('\n')

# Open a new CSV file for writing
csv_file_path = 'output.csv'
with open(csv_file_path, 'w', newline='') as f:
    writer = csv.writer(f)

    # Write the column headers
    writer.writerow(
        ['Page', 'Part Number', 'Description', 'Sheet', 'Y/P Bin', 'Sys Bin', 'Phy Count', 'Phy Y/P', 'Phy Tot', 'Comp',
         'Var', 'Unit Cost', 'Dollar Variance', 'Open R/O', 'Total Value'])

    # Loop over the lines in the text
    current_page = 1  # Initialize current_page to 1
    for line in lines[start_index:]:
        # Extract the page number from the line using a regular expression
        match = re.search(r'Page\s+(\d+)', line)
        if match:
            current_page = match.group(1)
            continue

        # Extract the fields from the line based on their character count
        if ':' in line:
            parts = line.split(':', 1)
            part_num = parts[0].strip()
            if len(parts) > 1:
                description = parts[1].strip()[:25]  # Use the first 40 characters of the description
            else:
                description = ''
        else:
            part_num = ''
            description = ''

        fields = [
            current_page,
            part_num,
            description,
            line[43:49].strip(),  # Sheet
            line[49:57].strip(),  # Y/P Bin
            line[57:70].strip(),  # Sys Bin
            line[70:80].strip(),  # Phy Count
            line[80:86].strip(),  # Phy Y/P
            line[86:93].strip(),  # Phy Tot
            line[93:100].strip(),  # Comp
            line[100:107].strip(),  # Var
            line[107:118].strip(),  # Unit Cost
            line[118:130].strip(),  # Dollar Variance
            line[130:138].strip(),  # Open R/O
            line[138:].strip()  # Total Value
        ]

        # Write the fields to the CSV file
        writer.writerow(fields)

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Define the regex pattern
pattern = r'PD3350R'

# Filter rows based on conditions
df = df[~df['Part Number'].str.contains(pattern, na=False, regex=True)]
df = df[~df['Sheet'].str.startswith(('='), na=False)]
df = df[~df['Phy Count'].str.startswith('=', na=False)]
df = df[~df['Total Value'].str.startswith('="', na=False)]

# Additional conditions
df = df[~((df['Sheet'].str.contains('Sheet', na=False)) &
          (df['Y/P Bin'].str.contains('Y/P Bi', na=False)) &
          (df['Sys Bin'].str.contains('n    Sys Bin', na=False)) &
          (df['Phy Count'].str.contains('Count', na=False)) &
          (df['Phy Y/P'].str.contains('Y/P', na=False)) &
          (df['Phy Tot'].str.contains('Tot', na=False)) &
          (df['Comp'].str.contains('Comp', na=False)) &
          (df['Var'].str.contains('Var', na=False)) &
          (df['Unit Cost'].str.contains('Cost', na=False)) &
          (df['Dollar Variance'].str.contains('Variance', na=False)) &
          (df['Open R/O'].str.contains('R/O', na=False)) &
          (df['Total Value'].str.contains('Value', na=False)))]

# Add an additional condition
df = df[~((df['Part Number'].isna()) &
          (df['Description'].isna()) &
          (df['Sheet'].isna()) &
          (df['Y/P Bin'].isna()) &
          (df['Sys Bin'].isna()) &
          (df['Phy Count'].isna()) &
          (df['Phy Y/P'].isna()) &
          (df['Phy Tot'].isna()) &
          (df['Comp'].isna()) &
          (df['Var'].isna()) &
          (df['Unit Cost'].isna()) &
          (df['Dollar Variance'].isna()) &
          (df['Open R/O'].isna()) &
          (df['Total Value'].isna()))]

df = df[~((df['Part Number'].isna()) &
          (df['Description'].isna()) &
          (df['Sheet'].isna()) &
          (df['Y/P Bin'].isna()) &
          (df['Sys Bin'].isna()) &
          (df['Comp'].isna()) &
          (df['Var'].isna()) &
          (df['Unit Cost'].str.contains('Unit', na=False)) &
          (df['Dollar Variance'].str.contains('Dollar', na=False)) &
          (df['Open R/O'].str.contains('Open', na=False)) &
          (df['Total Value'].str.contains('Total', na=False)))]

# Replace values in columns I, J, and K based on condition
df.loc[df['Phy Tot'].str.contains('Not', na=False), 'Phy Tot'] = 'Not Found In Count'
df.loc[df['Comp'].str.contains('Found I', na=False), 'Comp'] = ''
df.loc[df['Var'].str.contains('n Count', na=False), 'Var'] = ''

# Convert numeric columns to numbers
numeric_columns = ['Sheet', 'Phy Count', 'Phy Y/P', 'Phy Tot', 'Comp', 'Var', 'Open R/O']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(df[col])

# Convert currency columns to currency format
currency_columns = ['Unit Cost', 'Dollar Variance', 'Total Value']
for col in currency_columns:
    non_numeric = pd.to_numeric(df[col], errors='coerce').isna()
    df.loc[~non_numeric, col] = df.loc[~non_numeric, col].apply(lambda x: '${:,.2f}'.format(x) if pd.notnull(x) else x)

# Convert to XLSX and save the file with the same name as the uploaded file
xlsx_file_name = os.path.splitext(pdf_file_name)[0] + '.xlsx'
xlsx_file_path = os.path.join('/content', xlsx_file_name)
df.to_excel(xlsx_file_path, index=False)

print(f"CSV file saved: {csv_file_path}")
print(f"XLSX file saved: {xlsx_file_path}")


Saving 1ST UNIT VARIANCE DETAIL.PDF to 1ST UNIT VARIANCE DETAIL.PDF


<ipython-input-2-09dbe2dd6d42>:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce').fillna(df[col])


ValueError: ignored